In [1]:
import numpy as np

In [2]:
def read_img(path):
    arr = []
    with open(path, 'rb') as f:
        f.readline()
        f.readline()
        f.readline()
        while ((byte:=f.read(1)) and (len(arr)<10304)):
            arr.append(int.from_bytes(byte, "big"))
    return arr

def make_df():
    arr = []
    labels = []
    for i in range(1,41):
        for j in range(1,11):
            path = 's' + str(i) + '/' + str(j) + '.pgm'
            labels.append(i)
            arr.append(read_img(path))
    return np.array(arr), np.array(labels)

In [3]:
len(read_img('s1/1.pgm'))

10304

In [3]:
def train_test(matrix, labels):
    train_data = []
    test_data = []
    train_labels = []
    test_labels = []

    even = True
    for data, label in zip(matrix, labels):
        if even:
            train_data.append(data)
            train_labels.append(label)
        else:
            test_data.append(data)
            test_labels.append(label)
        even = not even

    return np.array(train_data), np.array(train_labels), np.array(test_data), np.array(test_labels)

In [4]:
X, y = make_df()

X_train, y_train, X_test, y_test = train_test(X,y)

In [12]:
X_train.shape

(200, 10304)

In [5]:
mu = np.mean(X_train, axis=0)
Z = X - 1*np.transpose(mu)
cov = np.cov(Z, rowvar=False, bias=True)
cov.shape

(10304, 10304)

In [16]:
print("Calculating mu: ..", end=' ')
mu = np.mean(X_train, axis=0)
print('Done.')
print("Calculating Z: ..", end=' ')
Z = X_train - 1*np.transpose(mu)
print('Done.')
print("Calculating covariance matrix: ..", end=' ')
# cov = np.cov(Z, rowvar=False,bias=True)
cov = (np.transpose(Z) @ Z) / X_train.shape[0]
print('Done.')

Calculating mu: .. Done.
Calculating Z: .. Done.
Calculating covariance matrix: .. Done.


In [17]:
cov.shape

(10304, 10304)

In [18]:
print("Calculating eigen vectors and values: ..", end=' ')
x,y = np.linalg.eig(cov)
print('Done.')

Calculating eigen vectors and values: .. Done.


In [19]:
print("Sorting eigen vectors and values: ..", end=' ')
eigen_values, eigen_vectors = zip(*sorted(zip(x,y), reverse=True)) # sorts eigen values and eigen vectors based on eigen values
eigen_vectors = np.array(eigen_vectors)
print('Done.')

Sorting eigen vectors and values: .. Done.


In [20]:
print("Calculating sum of eigen vectors: ..", end=' ')
total_variance = np.sum(eigen_values)
print('Done. -- eigen vectors shape: {}'.format(eigen_vectors.shape))

Calculating sum of eigen vectors: .. Done. -- eigen vectors shape: (10304, 10304)


In [37]:
total_variance

(16009799.676750034+5.169878828456423e-26j)

In [38]:
max(eigen_values)/total_variance

(0.18072160746605662-5.8358557330371405e-34j)

In [21]:
lim = 0
sum = 0
print("Finding top n eigen values to reach alpha ({}): ..".format(0.8), end=' ')
for i,val in enumerate(eigen_values):
    sum+=val
    if (sum/total_variance)>=0.8:
        lim = i+1
        break
print('Done. -- Found top {} eigen values reach {} variance.'.format(lim, sum/total_variance))

Finding top n eigen values to reach alpha (0.8): .. Done. -- Found top 36 eigen values reach (0.803622261197201-9.325975862915845e-34j) variance.


In [24]:
U = eigen_vectors[:,:lim]
# A = np.array([np.transpose(U)@x for x in X_train])
A = Z.dot(U)

In [42]:
def PCA(X, alpha):
    print("Calculating mu: ..", end=' ')
    mu = np.mean(X, axis=0)
    print('Done.')
    print("Calculating Z: ..", end=' ')
    Z = X - 1*np.transpose(mu)
    print('Done.')
    print("Calculating covariance matrix: ..", end=' ')
    # cov = np.cov(Z, rowvar=False,bias=True)
    cov = (np.transpose(Z) @ Z) / X.shape[0]
    print('Done.')
    print("Calculating eigen vectors and values: ..", end=' ')
    x,y = np.linalg.eig(cov)
    print('Done.')
    print("Sorting eigen vectors and values: ..", end=' ')
    eigen_values, eigen_vectors = zip(*sorted(zip(x,y), reverse=True)) # sorts eigen values and eigen vectors based on eigen values
    eigen_vectors = np.array(eigen_vectors)
    print('Done.')
    print("Calculating sum of eigen vectors: ..", end=' ')
    total_variance = np.sum(eigen_values)
    print('Done. -- eigen vectors shape: {}'.format(eigen_vectors.shape))
    lim = 0
    sum = 0
    print("Finding top n eigen values to reach alpha ({}): ..".format(alpha), end=' ')
    for i,val in enumerate(eigen_values):
        sum+=val
        if (sum/total_variance)>=alpha:
            lim = i+1
            break
    print('Done. -- Found top {} eigen values reach {} variance.'.format(lim, sum/total_variance))
    
    U = eigen_vectors[:,:lim]
    A = np.array([np.transpose(U)@x for x in X])

    return A
    

In [43]:
alpha = [0.8,0.85,0.9,0.95]
a = PCA(X_train, alpha[0])
a.shape

Calculating mu: .. Done.
Calculating Z: .. Done.
Calculating covariance matrix: .. Done.
Calculating eigen vectors and values: .. Done.
Sorting eigen vectors and values: .. Done.
Calculating sum of eigen vectors: .. Done. -- eigen vectors shape: (10304, 10304)
Finding top n eigen values to reach alpha (0.8): .. Done. -- Found top 36 eigen values reach (0.803622261197201-9.325975862915845e-34j) variance.


(200, 36)

In [ ]:
class PCA:
    def __init__(self, train_data, alpha):
        self.X = train_data
        self.alpha = alpha
        self.trained = False

    def find_values(self):
        print("Finding top n eigen values to reach alpha ({}): ..".format(self.alpha), end=' ')
        sum = 0
        r = 0
        for i,val in enumerate(self.eigen_values):
            sum+=val
            if (sum/self.total_variance)>=self.alpha:
                r = i+1
                break
        print('Done. -- Found top {} eigen values reach {} variance.'.format(lim, sum/self.total_variance))

        return r
    
    def update_alpha(self, alpha):
        self.alpha = alpha
        if self.trained:
            r = self.find_values()
            self.U = self.eigen_vectors[:,:r]
            self.A = np.array([np.transpose(self.U)@x for x in self.X])
    
    def train(self):
        print("Calculating mu: ..", end=' ')
        self.mu = np.mean(self.X, axis=0)
        print('Done.')
        print("Calculating Z: ..", end=' ')
        self.Z = self.X - 1*np.transpose(self.mu)
        print('Done.')
        print("Calculating covariance matrix: ..", end=' ')
        # cov = np.cov(Z, rowvar=False,bias=True)
        cov = (np.transpose(self.Z) @ self.Z) / self.Z.shape[0]
        print('Done.')
        print("Calculating eigen vectors and values: ..", end=' ')
        x,y = np.linalg.eig(cov)
        print('Done.')
        print("Sorting eigen vectors and values: ..", end=' ')
        self.eigen_values, eigen_vectors = zip(*sorted(zip(x,y), reverse=True)) # sorts eigen values and eigen vectors based on eigen values
        self.eigen_vectors = np.array(eigen_vectors)
        print('Done.')
        print("Calculating sum of eigen vectors: ..", end=' ')
        self.total_variance = np.sum(self.eigen_values)
        print('Done. -- eigen vectors shape: {}'.format(self.eigen_vectors.shape))
        r = self.find_values()
        self.U = self.eigen_vectors[:,:r]
        # self.A = np.array([np.transpose(self.U)@x for x in self.Z])
        self.A = self.Z.dot(U)
        self.trained = True

    def project_point(self, p):
        if not self.trained:
            return None
        return np.transpose(self.U)@(p-self.mu)